<a href="https://colab.research.google.com/github/hymgit0217/NeuSDF/blob/main/sdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

装载Google云盘

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


设置GPU

In [43]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

device: cuda


导入OBJ（Mesh）数据

In [44]:
!pip install trimesh rtree
import trimesh
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

mesh = trimesh.load("/content/drive/MyDrive/PermutoSDF2/bunny.obj")

if not mesh.is_watertight:
  print("Mesh is Fixing...")
  mesh.fill_holes()
  trimesh.repair.fix_normals(mesh)
  trimesh.repair.fix_inversion(mesh)
else:
  print("Mesh is Good.")

min_bound, max_bound = mesh.bounds
print(f"the range of mesh: min_bound: {min_bound}, max_bound:{max_bound}")
center = (min_bound + max_bound) / 2 # 用这个办法计算包围盒中心
print(f"the center of mesh: {center}")
size = max_bound - min_bound
print(f"the size of mesh: {size}")

# mesh->[-1, 1] # 确认一下这段代码的正确与否
scale = 2.0 / max(size)
mesh.apply_translation(-center)
mesh.apply_scale(scale)

min_bound, max_bound = mesh.bounds
print(f"the range of new mesh: min_bound: {min_bound}, max_bound:{max_bound}")
center = (min_bound + max_bound) / 2
print(f"the center of new mesh: {center}")
size = max_bound - min_bound
print(f"the size of new mesh: {size}")

Mesh is Fixing...
the range of mesh: min_bound: [-0.09438042  0.0333099  -0.06167917], max_bound:[0.0607788  0.18699602 0.05871464]
the center of mesh: [-0.01680081  0.11015296 -0.00148226]
the size of mesh: [0.15515922 0.15368612 0.12039381]
the range of new mesh: min_bound: [-1.         -0.99050588 -0.7759372 ], max_bound:[1.         0.99050588 0.7759372 ]
the center of new mesh: [0. 0. 0.]
the size of new mesh: [2.         1.98101176 1.55187439]


可视化导入的Mesh

In [45]:
import plotly.graph_objects as go

x, y, z = mesh.vertices.T
i, j, k = mesh.faces.T

fig = go.Figure(data=[go.Mesh3d(
    x=x, y=y, z=z,
    i=i, j=j, k=k,
    color='lightblue',
    opacity=0.5
)])

fig.show()

封装数据集

In [46]:
import numpy as np
import torch
import trimesh

query_points = np.random.uniform(low=-2, high=2, size=(50000, 3)) # >=100_000的RAM会报错

# 计算SDF值
_, real_distances, _ = trimesh.proximity.closest_point(mesh, query_points) # 分别返回closest_point，distance，triangle_id
inside = mesh.contains(query_points)
real_sdfs_numpy = real_distances * np.where(inside, -1, 1)

points = torch.tensor(query_points, dtype=torch.float32).to(device)
real_sdfs = torch.tensor(real_sdfs_numpy, dtype=torch.float32).to(device)

dataset = TensorDataset(points, real_sdfs)
train_ratio = 0.9
train_size = int(len(dataset)*train_ratio)
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=100, shuffle=False)

SDF的MLP定义

In [47]:
import torch

class SDF(torch.nn.Module):
  def __init__(self, num_frequencies=6):
    super(SDF, self).__init__()
    input_shape = 3 + 3 * 2 * num_frequencies
    self.num_frequencies = num_frequencies
    self.linear1 = torch.nn.Linear(input_shape, 128)
    self.linear2 = torch.nn.Linear(128, 128)
    self.linear3 = torch.nn.Linear(128, 1)
    self.relu = torch.nn.ReLU()

  def forward(self, xyz):
    h = self.position_encoding(xyz, self.num_frequencies)
    h = torch.cat([xyz, h], dim=-1)
    h = self.relu(self.linear1(h))
    h = self.relu(self.linear2(h))
    sdf = self.linear3(h)
    return sdf

  # 计算SDF及其梯度值
  # PermutoSDF：https://github.com/RaduAlexandru/permuto_sdf/blob/main/permuto_sdf_py/models/models.py#L199
  def get_sdf_and_gradient(self, xyz):
    xyz.requires_grad_(True)
    sdf = self.forward(xyz)
    grad_output = torch.ones_like(sdf)
    gradients = torch.autograd.grad(
        outputs=sdf,
        inputs=xyz,
        grad_outputs=grad_output,
        create_graph=True, # 创建计算图，用于高阶求导，默认False
        retain_graph=True, # 保留计算图，默认是create_graph值
    )
    return sdf, gradients[0]

  # 高频位置编码（NeRF）
  def position_encoding(self, xyz, num_frequencies=6):
    N, D = xyz.shape
    freq_bands = 2.0 ** torch.arange(num_frequencies, device=xyz.device) * torch.pi
    xb = xyz[..., None] * freq_bands
    sin = torch.sin(xb)
    cos = torch.cos(xb)
    encoded = torch.stack([sin, cos], dim=-1).flatten(-2)
    encoded = encoded.view(N, D * 2 * num_frequencies)

    return encoded

# 实例化神经SDF
sdf = SDF().to(device)

SDF的GT值

In [48]:
# 计算sdf的真实值函数（待实现）
def compute_sdf_gt(xyz):
  return None

损失函数

In [49]:
import torch

# Eikonal损失函数
# PermutoSDF：https://github.com/RaduAlexandru/permuto_sdf/blob/main/permuto_sdf_py/utils/permuto_sdf_utils.py#L49
def eikonal_loss(sdf_gradients):
    gradient_error = (torch.linalg.norm(sdf_gradients.reshape(-1, 3), ord=2, dim=-1) - 1.0) ** 2
    return gradient_error.mean()

训练和评估SDF（从Mesh）

In [50]:
import torch

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(sdf.parameters())

epochs = 300

for epoch in range(epochs):
  train_loss = 0
  loss_distance = 0
  loss_eikonal = 0
  loss = 0
  sdf.train()
  for batch_point, batch_sdf in train_dataloader: # 小批量梯度下降效果更好
    optimizer.zero_grad()
    sdf_pred, gradients = sdf.get_sdf_and_gradient(batch_point)
    distances = criterion(sdf_pred.squeeze(), batch_sdf)
    eikonal = eikonal_loss(gradients) # Eikonal损失
    loss = distances + eikonal
    loss.backward()
    optimizer.step()
    loss_distance += distances.item()
    loss_eikonal += eikonal.item()
    train_loss += loss.item()
  test_loss = 0
  loss_distance_test = 0
  loss_eikonal_test = 0
  loss_test = 0
  sdf.eval()
  for batch_point, batch_sdf in test_dataloader:
    sdf_pred, gradients = sdf.get_sdf_and_gradient(batch_point)
    distances = criterion(sdf_pred.squeeze(), batch_sdf)
    eikonal = eikonal_loss(gradients)
    loss_test = distances + eikonal
    loss_distance_test += distances.item()
    loss_eikonal_test += eikonal.item()
    test_loss += loss_test.item()
  if (epoch+1)%10 == 0:
    print(f'[{epoch+1}/{epochs}] train_loss:{train_loss/len(train_dataloader):.3f}  test_loss:{test_loss/len(test_dataloader):.3f}')
    print(f'         loss_distance:{loss_distance/len(train_dataloader):.3f}  test_loss_distance:{loss_distance_test/len(test_dataloader):.3f}')
    print(f'         loss_eikonal:{loss_eikonal/len(train_dataloader):.3f}  test_loss_eikonal:{loss_eikonal_test/len(test_dataloader):.3f}')

[10/300] train_loss:0.022  test_loss:0.022
         loss_distance:0.004  test_loss_distance:0.004
         loss_eikonal:0.018  test_loss_eikonal:0.017
[20/300] train_loss:0.016  test_loss:0.016
         loss_distance:0.003  test_loss_distance:0.003
         loss_eikonal:0.013  test_loss_eikonal:0.013
[30/300] train_loss:0.014  test_loss:0.016
         loss_distance:0.002  test_loss_distance:0.002
         loss_eikonal:0.012  test_loss_eikonal:0.013
[40/300] train_loss:0.014  test_loss:0.015
         loss_distance:0.002  test_loss_distance:0.002
         loss_eikonal:0.012  test_loss_eikonal:0.013
[50/300] train_loss:0.014  test_loss:0.014
         loss_distance:0.002  test_loss_distance:0.002
         loss_eikonal:0.012  test_loss_eikonal:0.012
[60/300] train_loss:0.012  test_loss:0.013
         loss_distance:0.002  test_loss_distance:0.002
         loss_eikonal:0.010  test_loss_eikonal:0.011
[70/300] train_loss:0.011  test_loss:0.011
         loss_distance:0.002  test_loss_distance:0.

从神经SDF中提取Mesh

In [51]:
import numpy as np
from skimage import measure
import trimesh
import torch

# 创建3D网格
grid_size = 128
x, y, z = np.linspace(-2, 2, grid_size), np.linspace(-2, 2, grid_size), np.linspace(-2, 2, grid_size)
X, Y, Z = np.meshgrid(x, y, z)
points = np.stack([X, Y, Z], axis=-1).reshape(-1, 3) # 笛卡尔积
points_tensor = torch.tensor(points, dtype = torch.float32).to(device)
with torch.no_grad():
  sdf_values_tensor = sdf(points_tensor)
  sdf_values_numpy = sdf_values_tensor.squeeze().cpu().numpy()
volume = sdf_values_numpy.reshape(grid_size, grid_size, grid_size)

# 使用Marching Cubes提取等值面
verts, faces, normals, _ = measure.marching_cubes(volume, level=0.0)

def transform_vertices(verts, bounds, resolution):
    scale = (bounds[1] - bounds[0]) / (resolution - 1)
    return verts * scale + bounds[0]

verts = transform_vertices(verts, bounds=[-2, 2], resolution=grid_size)

# 创建Mesh
mesh_output = trimesh.Trimesh(vertices=verts, faces=faces, vertex_normals=normals)

# 导出Mesh
mesh_output.export("/content/neural_sdf_bunny.obj")

'# https://github.com/mikedh/trimesh\nv -0.96197355 -0.61417317 0.04724407\nv -0.96062994 -0.61607182 0.04724407\nv -0.96062994 -0.61417317 0.04340386\nv -0.96062994 -0.61417317 0.07863021\nv -0.96091175 -0.58267713 -0.01574802\nv -0.96062994 -0.58292627 -0.01574802\nv -0.96062994 -0.58267713 -0.01613224\nv -0.97342706 -0.58267713 0.01574802\nv -0.96062994 -0.60087264 0.01574802\nv -0.97362530 -0.58267713 0.04724407\nv -0.96062994 -0.61416125 0.07874012\nv -0.97260535 -0.58267713 0.07874012\nv -0.97039855 -0.58267713 0.11023617\nv -0.96062994 -0.60965323 0.11023617\nv -0.96704721 -0.58267713 0.14173222\nv -0.96062994 -0.60017300 0.14173222\nv -0.96368146 -0.58267713 0.17322826\nv -0.96062994 -0.59097099 0.17322826\nv -0.96198118 -0.58267713 0.20472431\nv -0.96062994 -0.58661878 0.20472431\nv -0.96253061 -0.58267713 0.23622036\nv -0.96062994 -0.58822131 0.23622036\nv -0.96062994 -0.58267713 0.25795722\nv -0.96818364 -0.55118108 -0.04724407\nv -0.96062994 -0.55914390 -0.04724407\nv -0.96

可视化训练后的SDF

In [52]:
import plotly.graph_objects as go

x, y, z = mesh_output.vertices.T
i, j, k = mesh_output.faces.T

fig = go.Figure(data=[go.Mesh3d(
    x=x, y=y, z=z,
    i=i, j=j, k=k,
    color='lightblue',
    opacity=0.5
)])

fig.show()

绘制2D切片图（效果很差）

In [53]:
# import plotly.graph_objects as go

# # 获取归一化边界框
# bounds = mesh.bounds
# x_range = [bounds[0][0], bounds[1][0]]
# y_range = [bounds[0][1], bounds[1][1]]
# z_range = [bounds[0][2], bounds[1][2]]
# print(f"归一化边界框: x={x_range}, y={y_range}, z={z_range}")

# # 创建 3D 网格
# grid_res = 64  # 64x64x64，适合 Colab 内存
# x = np.linspace(x_range[0], x_range[1], grid_res)
# y = np.linspace(y_range[0], y_range[1], grid_res)
# z = np.linspace(z_range[0], z_range[1], grid_res)
# xx, yy, zz = np.meshgrid(x, y, z)
# points = np.stack([xx.ravel(), yy.ravel(), zz.ravel()], axis=-1)

# # 预测 SDF
# points_tensor = torch.tensor(points, dtype=torch.float32).to(device)
# with torch.no_grad():
#     sdf_values = sdf(points_tensor).cpu().numpy().reshape(grid_res, grid_res, grid_res)

# # 检查 SDF 范围
# sdf_min, sdf_max = sdf_values.min(), sdf_values.max()
# print(f"3D SDF 范围: [{sdf_min:.3f}, {sdf_max:.3f}]")

# # 绘制 3D 等值面
# fig = go.Figure(data=[
#     go.Isosurface(
#         x=xx.flatten(),
#         y=yy.flatten(),
#         z=zz.flatten(),
#         value=sdf_values.flatten(),
#         isomin = np.min(sdf_values) * 0.5,
#         isomax = np.max(sdf_values) * 0.5,
#         colorscale='RdBu',
#         showscale=True,
#         opacity=1,
#         surface_count=1,
#         caps=dict(x_show=False, y_show=False, z_show=False)
#     )
# ])
# fig.update_layout(
#     title="3D SDF 等值面 (SDF=0)",
#     scene=dict(
#         xaxis_title='X',
#         yaxis_title='Y',
#         zaxis_title='Z',
#         aspectmode='cube'
#     ),
#     margin=dict(l=0, r=0, b=0, t=50)
# )
# fig.show()

2、采样球面的SDF的真实值（可选择不使用采样值）

In [54]:
# import torch
# from torch.utils.data import TensorDataset, random_split, DataLoader

# xyz = (4*torch.rand(1000, 3) - 2).to(device) # 均匀分布[-2, 2]
# sphere_sdf = (xyz[:, 0]**2 + xyz[: ,1]**2 + xyz[: ,2]**2)**0.5 - 1 # 半径为1的球面的SDF函数

# dataset = TensorDataset(xyz, sphere_sdf)

# train_ratio = 0.7
# train_size = int(train_ratio*len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
# train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=100, shuffle=True)

3、训练SDF

In [55]:
# import torch

# criterion = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(sdf.parameters())

# epochs = 100

# sdf.train()
# for epoch in range(epochs):
#   train_loss = 0
#   loss = 0
#   for data in train_dataloader:
#     inputs, real_sdfs = data
#     outputs = sdf(inputs)
#     optimizer.zero_grad()
#     loss = criterion(outputs.flatten(), real_sdfs)
#     loss.backward()
#     optimizer.step()
#     train_loss += loss.item()
#   if (epoch+1)%10 == 0:
#     print(f'[{epoch+1}/{epochs}] train_loss:{train_loss/len(train_dataloader):.3f}')

4、评估SDF

In [56]:
# with torch.no_grad():
#   test_loss = 0
#   loss = 0
#   for data in test_dataloader:
#     inputs, real_sdfs = data
#     outputs = sdf(inputs)
#     loss = criterion(outputs.flatten(), real_sdfs)
#     test_loss += loss.item()
#   print(f'test_loss:{test_loss/len(test_dataloader):.3f}')

绘制SDF的二维切片图

In [57]:
# import numpy as np
# import matplotlib.pyplot as plt
# import torch

# z0 = 0.0 # 绘制的是z=0的切片
# grid_res = 512
# x = np.linspace(-2, 2, grid_res)
# y = np.linspace(-2, 2, grid_res)
# xx, yy = np.meshgrid(x, y)
# zz = np.full_like(xx, z0)
# points = np.stack([xx, yy, zz], axis=-1).reshape(-1, 3)

# points_tensor = torch.tensor(points, dtype=torch.float32).to(device)
# with torch.no_grad():
#   sdf_values = sdf(points_tensor).cpu().numpy().reshape((grid_res, grid_res))

# plt.figure(figsize=(6, 6))
# plt.contour(xx, yy, sdf_values, levels=[0], colors='black') # sdf=0绘制为黑色
# plt.imshow(sdf_values, extent=[-2,2,-2,2], origin='lower', cmap='coolwarm', alpha=1,
#            vmin=-1.0, vmax=1.0)
# plt.colorbar()
# plt.title("2D SDF Slice")
# plt.show()